In [ ]:
import psycopg2
import pandas as pd
from tqdm import tqdm
import time
import warnings

warnings.filterwarnings("ignore")

host = 'awesome-hw.sdsc.edu'
database = 'postgres'
user = 'ag_class'
password = 'W9TVptLu'

In [ ]:
connection = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )

cursor = connection.cursor()

In [ ]:
cursor.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'usnewspaper';")
rows = cursor.fetchall()

for row in rows:
    print(row)

In [ ]:
col_names = [col[0] for col in rows]

df = pd.DataFrame(columns = col_names)
df

In [ ]:
entries = []

In [ ]:
# There is a around 19.7 Million Rows
# I used (SELECT Count(*) FROM usnewspaper)
# WITH BATCH SIZE 1000 FOR 100000, took 28.15s
# WITH BATCH SIZE 5000 FOR 100000, took 28.37s
# WITH BATCH SIZE 10000 FOR 100000, took 34.51s
# WITH BATCH SIZE 5000 for 500000, took 155.12s
# WITH BATCH SIZE 10000 for 1000000, took 407.56

In [ ]:
start_time = time.time()

cursor.execute("SELECT * FROM usnewspaper WHERE EXTRACT(YEAR FROM publishdate) IN ('2018', '2019') LIMIT 1000000")
batch_size = 10000
#rows = cursor.fetchall()

while True:
    results = cursor.fetchmany(batch_size)

    if not results:
        break

    # Iterate over the results using tqdm to track progress
    for row in tqdm(results, total=batch_size):
        x = row
        new_row = pd.Series(x, index=df.columns)
        entries.append(new_row)
        
end_time = time.time()

print(end_time - start_time)

In [ ]:
pd.DataFrame(entries)

In [ ]:
cursor.execute("SELECT * FROM usnewspaper LIMIT 100000")
batch_size = 1000

#rows = cursor.fetchall()

while True:
    results = cursor.fetchmany(batch_size)

    if not results:
        break

    # Iterate over the results using tqdm to track progress
    for row in tqdm(results, total=batch_size):
        x = row
        new_row = pd.Series(x, index=df.columns)
        df = df.append(new_row, ignore_index=True)

In [ ]:
df

In [ ]:
df.to_csv('output.csv', index = False)

In [ ]:
cursor.close()
connection.close()